In [ ]:
# default_exp tools.image_size_calculator

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import logging
import argparse
import logging.handlers
import sys
from PIL import Image as PILImage
from os import listdir
from os.path import join, isfile, isdir, splitext

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

logger = logging.getLogger(__name__)

In [ ]:
# export

FILE_FILTER = ['.DS_Store']
IMAGE_EXTENSIONS = ['.jpg']

# Image Size Calculator

> Calculates image sizes in a given folder and subfolders. Summarize unique image sizes at the end.

In [ ]:
# export


class ImageSizeCalculator:
    """
    Calculates image sizes in a given folder and subfolders. Summarize unique image sizes at the end.
    `path`: the folder to process
    """

    def __init__(self, path):
        self.path = path

    def calculate(self):
        """
        The main logic.
        """

        images = scan_folder(self.path)
        all_images = len(images)
        unique_sizes = {}

        for index, image in enumerate(images):
            width, height = get_image_size(image)
            logger.info("{} / {} - Handle Image {} with size {}x{}".format(
                index + 1,
                all_images,
                image,
                width,
                height,
            ))
            unique_sizes["{}x{}".format(width, height)] = True

        return unique_sizes.keys()

In [ ]:
show_doc(ImageSizeCalculator.calculate)

## Helper Methods

In [ ]:
# export


def configure_logging(logging_level=logging.INFO):
    """
    Configures logging for the system.

    :param logging_level: The logging level to use.
    """
    logger.setLevel(logging_level)

    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging_level)

    logger.addHandler(handler)

In [ ]:
# export


def get_image_size(fname):
    """
    Calculates image size of a given image file.
    `fname`: the file path
    return: the image width and height
    """
    img = PILImage.open(fname)
    w, h = img.size
    return w, h

In [ ]:
# export


def scan_folder(folder):
    """
    Scans a folder and subfolders for image content.
    `folder`: the folder to scan
    returns: a list of paths to images found
    """
    images = []

    contents = listdir(folder)
    for content in contents:
        if content in FILE_FILTER:
            continue
        path = join(folder, content)
        if isdir(path):
            result = scan_folder(path)
            for entry in result:
                images.append(entry)
        elif isfile(path):
            if not splitext(content)[1] in IMAGE_EXTENSIONS:
                continue
            images.append(path)
    return images

## Run from command line

To run the image size calculator from command line, use the following command:
`python -m mlcore.tools.image_size_calculator [parameters]`

The following parameters are supported:
- `[folder]`: The folder to scan.

In [ ]:
# export


if __name__ == '__main__' and '__file__' in globals():
    configure_logging()

    parser = argparse.ArgumentParser()
    parser.add_argument("folder", help="The folder to scan.")
    args = parser.parse_args()

    calculator = ImageSizeCalculator(args.folder)
    sizes = calculator.calculate()
    logger.info("Unique Image Sizes:")
    for size in sizes:
        logger.info(size)

    print('FINISHED!!!')
